### Importing the packages

In [25]:
import numpy as np

import torch
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
from torchvision.models import resnet34
from torch.utils.data import DataLoader

from sklearn.metrics import confusion_matrix, f1_score
from tqdm import tqdm

from numpy.ma.core import ceil
from scipy.spatial import distance #distance calculation
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import accuracy_score #scoring
import matplotlib.pyplot as plt
from matplotlib import animation, colors


### Loading Data

In [26]:
transform = transforms.Compose([
    transforms.Resize((64, 64)), # Resize to 224x224 (height x width)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])
])

In [27]:
batch_size = 32
#drop_last=True
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
train_dataloader = DataLoader(train_data, batch_size=batch_size,shuffle=True )

#loading the test data
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)
test_dataloader = DataLoader(test_data,batch_size=batch_size, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Setting up the Feature Extractor

In [29]:
feature_extractor = resnet34(weights="DEFAULT")
num_features = feature_extractor.fc.in_features

for param in feature_extractor.parameters():
  param.requires_grad = False

feature_extractor.fc = nn.Identity()
feature_extractor = feature_extractor.to(device)

### Finding Centers using Minibatch K-Means

In [ ]:
kmeans =  MiniBatchKMeans(n_clusters=20, max_iter=100, random_state=0, batch_size=32)

for x_train, y_train in tqdm(train_dataloader, desc=f"Training", colour="blue"):
    x_train, y_train = x_train.to(device), y_train.to(device)
    x = feature_extractor(x_train)
    x = x.cpu()
    kmeans = kmeans.partial_fit(x)

Training:   0%|          | 0/1563 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
Training:  32%|███▏      | 497/1563 [00:10<00:23, 45.75it/s]

In [19]:
kmeans.cluster_centers_

(20, 512)

In [31]:
centers =  torch.from_numpy(kmeans.cluster_centers_.copy())
centers.a

tensor([[0.0000e+00, 3.8573e+00, 6.7269e+00,  ..., 2.2539e+00, 0.0000e+00,
         0.0000e+00],
        [1.3931e+00, 1.8627e+00, 1.8067e+00,  ..., 1.8352e+00, 1.1641e+00,
         1.0276e+00],
        [1.5506e+00, 6.6554e-01, 9.4351e-01,  ..., 1.0590e+00, 4.6419e-02,
         0.0000e+00],
        ...,
        [3.9156e-01, 3.4024e-01, 2.5706e-01,  ..., 3.9279e-01, 5.6235e-01,
         1.0063e+00],
        [1.2064e+00, 2.3570e-04, 4.5726e-02,  ..., 8.7305e-04, 2.2045e+00,
         7.1516e-02],
        [1.2441e+00, 3.3544e-01, 1.9293e-01,  ..., 5.9881e-01, 1.0056e+00,
         0.0000e+00]], dtype=torch.float64)

### RBF Layer

In [ ]:
class RBF_Layer:
    def __init__(self, n_inputs, n_neurons, centers):
        self.weights = torch.randn(n_inputs, n_neurons)
        self.biases = torch.zeros((1, n_neurons))
        self.weights = self.weights.to(device)
        self.biases = self.biases.to(device)
        self.prev_wchange = torch.zeros((n_inputs, n_neurons))
        self.prev_bchange = torch.zeros((1, n_neurons))
        self.prev_wchange = self.prev_wchange.to(device)
        self.prev_bchange = self.prev_bchange.to(device)

    
    def forward(self, inputs):
        self.inputs = inputs
        self.inputs = self.inputs.to(device)
        return torch.matmul(self.inputs, self.weights) + self.biases

    def backward(self, output_error):
        # calculating errors
        self.inputs_error = torch.matmul(output_error, self.weights.T)
        self.inputs_error = self.inputs_error.to(device)
        self.weights_grad = torch.matmul(self.inputs.T, output_error)
        self.weights_grad = self.weights_grad.to(device)
        self.biases_grad = torch.sum(output_error, axis=0, keepdims=True)
        self.biases_grad =  self.biases_grad.to(device)
        return self.inputs_error
